In [8]:
import re
import requests
import pickle
import spacy
import httplib2
from urllib.request import urlopen
from bs4 import BeautifulSoup, SoupStrainer
from newspaper import Article

In [2]:
response = requests.get(url='https://www.foxnews.com/sitemap.xml?type=news')
soup=BeautifulSoup(response.content)
aa = soup.find('url')
test_url = str(aa.find('loc'))[5:-6]
print(test_url)

https://www.foxnews.com/us/washington-sheriffs-deputy-stabbed-in-head-while-trying-to-arrest-rape-suspect


In [4]:
article = Article(test_url)
article.download()
article.parse()
text=article.text

In [43]:
http = httplib2.Http()
status, response = http.request('https://www.foxnews.com/sports')

extensions = []
for link in BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        if str(link['href'])[0:1] == '/':
            extensions.append(str(link['href']))
fullFoxArticles = []
for linkExtension in extensions:
    url = 'https://www.foxnews.com' + linkExtension
    article = Article(url)
    article.download()
    article.parse()
    text=article.text
    fullFoxArticles.append(text)

In [27]:
print(len(fullArticles))

225


In [40]:
http = httplib2.Http()
status, response = http.request('https://abcnews.go.com/Sports/Olympics')

urls = []
for link in BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        if '/GMA/' in str(link['href']) or '/Sports/' in str(link['href']):
            if str(link['href']) not in urls and str(link['href']) != '/Sports/Olympics':
                urls.append(str(link['href']))

status, response = http.request('https://abcnews.go.com/Sports')

for link in BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        if '/GMA/' in str(link['href']) or '/Sports/' in str(link['href']):
            if str(link['href']) not in urls and str(link['href']) != '/Sports/Olympics':
                urls.append(str(link['href']))
                
status, response = http.request('https://abcnews.go.com/Health')

for link in BeautifulSoup(response,'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        if '.com/Health/' in str(link['href']):
            if str(link['href']) not in urls:
                urls.append(str(link['href']))
       
       
fullABCArticles = []
for link in urls:
    article = Article(link)
    article.download()
    article.parse()
    text=article.text
    fullABCArticles.append(text)

In [42]:
print(len(fullABCArticles))

141
